In [ ]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

# 1. Wczytywanie danych

In [ ]:
file_path = "data.txt"
data = pd.read_csv(file_path, delimiter= "\t")  # Pomijamy opisowe linie
# Wyświetlenie pierwszych wierszy
print(data.head())


# Wyświetlenie wszystkich wykresów punktowych oddzielnie

In [ ]:
variables = data.columns.tolist()
#ariables.remove("Pct.BF")  # Usuwamy "Pct.BF", ponieważ jest zmienną zależną

# Tworzenie wykresów punktowych
for variable in variables:
    plt.figure(figsize=(8, 6))
    plt.scatter(data[variable], data["Pct.BF"], alpha=0.7)
    plt.title(f"Wykres punktowy: Pct.BF vs {variable}")
    plt.xlabel(variable)
    plt.ylabel("Pct.BF")
    plt.grid(True)
    plt.show()  # Wyświetlenie każdego wykresu osobno

In [ ]:
fig, axs = plt.subplots(4, 4, figsize=(12, 12))
i = 0

for i, (name, X) in enumerate(data.items()):
    row = i // 4
    col = i % 4
    
    axs[row][col].scatter(X, data['Pct.BF'])
    axs[row][col].set_xlabel(name)
    axs[row][col].set_ylabel('Pct.BF')

fig.tight_layout()
fig.show()

In [ ]:
# Obliczenie macierzy korelacji
correlation_matrix = data.corr()

# Wyświetlenie korelacji z "Pct.BF"
correlations_with_bodyfat = correlation_matrix["Pct.BF"].sort_values(ascending=False)
print("Korelacje ze zmienną 'Pct.BF':\n", correlations_with_bodyfat)


# model ze wszystkimi zmiennymi:

In [ ]:
# Lista wszystkich zmiennych predykcyjnych (pomijając zmienną zależną "Pct.BF")
predictors = [col for col in data.columns if col != "Pct.BF"]

# Przygotowanie danych
X = sm.add_constant(data[predictors])  # Wszystkie zmienne predykcyjne + stała
y = data["Pct.BF"]  # Zmienna zależna

# Tworzenie pełnego modelu
model = sm.OLS(y, X).fit()

# Raport modelu regresji (wyniki statystyczne)
print(model.summary())


# LASSO

In [ ]:
X = data.drop(columns=["Pct.BF"])
y = data["Pct.BF"]

# Podział na dane treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Skalowanie danych
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#Forawrd selection:
def forward_selection(X, y):
    remaining_features = list(X.columns)
    selected_features = []
    current_score, best_new_score = float('-inf'), float('-inf')

    while remaining_features:
        scores_with_candidates = []
        for candidate in remaining_features:
            features_to_test = selected_features + [candidate]
            model = LinearRegression()
            score = cross_val_score(model, X[features_to_test], y, cv=5, scoring='neg_mean_squared_error').mean()
            scores_with_candidates.append((score, candidate))

        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if best_new_score > current_score:
            remaining_features.remove(best_candidate)
            selected_features.append(best_candidate)
            current_score = best_new_score
        else:
            break

    return selected_features

# Forward Selection
selected_features = forward_selection(pd.DataFrame(X_train_scaled, columns=X.columns), y_train)
print("Wybrane zmienne przez Forward Selection:", selected_features)

# --- LASSO ---
# Wybrane zmienne przez Forward Selection
X_train_selected = X_train_scaled[:, [X.columns.get_loc(f) for f in selected_features]]
X_test_selected = X_test_scaled[:, [X.columns.get_loc(f) for f in selected_features]]

# LASSO z walidacją krzyżową
lasso_cv = LassoCV(cv=5, random_state=42).fit(X_train_selected, y_train)
print("Optymalne alpha LASSO:", lasso_cv.alpha_)
print("Współczynniki LASSO:")
print(pd.Series(lasso_cv.coef_, index=selected_features))

# Ocena na danych testowych
y_pred = lasso_cv.predict(X_test_selected)
mse = mean_squared_error(y_test, y_pred)
print("Błąd MSE na danych testowych:", mse)